In [65]:
print("-----------------initializing------------------------")
print("                    created by - GH 5기 권기남 백승렬")
print("                                                     ")
print("Designed to crawl Naver hourly real-time data")
print("Saved destionation: GH bigquery NaverCrawler.Trending20")
print("Starting from 2019-01-01")
print("Each operation updates database to date of operation")
print("Time taken = 3 minutes per day")

-----------------initializing------------------------
                    created by - GH 5기 권기남 백승렬
                                                     
Designed to crawl naver hourly real-time data
Saved destionation: GH bigquery NaverCrawler.Trending20
Starting from 2019-01-01
Each operation updates database to date of operation
Time taken = 3 minutes per day


## import

In [87]:
import selenium
from bs4 import BeautifulSoup as bs
import lxml
from selenium import webdriver
import time
import requests
from selenium.webdriver.common.keys import Keys
from pprint import pprint
from datetime import datetime

import pandas as pd
import pandas_gbq as gbq
import numpy as np

from google.cloud import bigquery

from google.oauth2 import service_account

## Connect to bigquery

In [94]:
### 'NaverCrawler-f54b101166fd.json' located in GH drive

credential_json = 'your_bigquery_json_file.json' #downloaded and located in your path

credentials = service_account.Credentials.from_service_account_file(credential_json)

project_id = 'bigquery_project_id'

client = bigquery.Client(credentials= credentials, project=project_id)

table_name = 'your_table_name'

## connect to chromedriver/ setup for crawling

In [95]:
options = webdriver.ChromeOptions()

options.add_argument('headless')

options.add_argument('window-size=1920x1080')

path = 'your\\chromedriver\\path\\chromedriver.exe' # Set path to chromedriver. linux/ mac does not have .exe

## prepare columns

In [96]:
colnames = ['Age', 'Datetime', 'Rank1', 'Rank2', 'Rank3', 'Rank4', 'Rank5', 'Rank6', 'Rank7', 'Rank8', 'Rank9', 'Rank10',
            'Rank11', 'Rank12', 'Rank13', 'Rank14', 'Rank15', 'Rank16', 'Rank17', 'Rank18', 'Rank19', 'Rank20']
times = ['00','01','02', '03','04', '05', '06', '07', '08', '09','10','11','12','13','14','15','16','17','18','19','20','21','22','23']
table_list = ['all', '10s', '20s', '30s', '40s', '50s']

##  function to get specific day's all age range's trending keywords (top 20) for each hour and append to database

In [105]:
def daily_update(input_date):
    daily = []
    
    ### repeat for each hour (time)

    for time in times:
        
        ### connect to chromedriver
        
        driver = webdriver.Chrome(path, options=options)

        ### connect to trending keywords in specific input date and time, and parse
        
        driver.get("https://datalab.naver.com/keyword/realtimeList.naver?datetime={}T{}:00:00".format(input_date, time))
        
        driver.implicitly_wait(4)

        naver_main = driver.page_source

        parsed_naver = bs(naver_main, 'lxml')

        ### for each age range - all, 10s to 50s
        
        for age in table_list:
            tmplist = [age, input_date+" "+time] #first sections of row that would look [age, date time, rank1, rank2, ...rank 20] 
            table_all = parsed_naver.find("div", {"data-age":"{}".format(age)}) #get specific age of parsed info
            for i in table_all.find_all("span", {"class": "title"}):
                tmplist.append(i.text) #append rank1 to rank20 to previously made list
            daily.append(tmplist) #append entire row
        driver.quit()
    
    df = pd.DataFrame(daily, columns = colnames) ### create dataframe of 1 month x all age range x all time in day
    
    gbq.to_gbq(df, table_name, project_id, if_exists='append', credentials = credentials) ### append df to DB
    
    print("completed " + input_date)

## get year/ month/ date of last update made to DB

In [106]:
sql_line = "select Datetime from NaverCrawler.Trending20 order by Datetime desc limit 1" 

query_job = client.query(sql_line)

results = query_job.result()

df = pd.DataFrame(results)

tmp = str(df[0])

year = int(tmp[6:10])
month = int(tmp[11:13])
date_of_month = int(tmp[14:16])

## get date_list from last updated date in DB to current

In [107]:
from datetime import timedelta, date

def daterange(start_date, end_date): ## gets range of dates between two days (input type = date)
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

now = datetime.now()
        
start_date = date(int(year), int(month), int(date_of_month)) # input from above cell
start_date = start_date + timedelta(days = 1) #start date from one day after the last record in DB
end_date = now.date() 

date_list = []
for single_date in daterange(start_date, end_date): #for every date in between start_date and end_date, fill in date_list
    date_list.append(single_date.strftime("%Y-%m-%d"))

## for every date to be updated, update DB
## each day costs approx. 3 minutes (local computer)

In [ ]:
for date in date_list:
    daily_update(date)




0it [00:00, ?it/s]


1it [00:15, 15.64s/it]

completed 2019-09-20





0it [00:00, ?it/s]


1it [00:05,  5.70s/it]

completed 2019-09-21


## errors and limitations:
### operating for 30 date causes next operation to crash
### Occasional crash in calling chromedriver found
### Bug found in 0it and 1it, but not sure what 0it means. There should be one iteration to bigquery (data, however, is gathered without error)
### Bug found "ReadTimeout: HTTPSConnectionPool(host='bigquery.googleapis.com', port=443): Read timed out. (read timeout=60)"